In [1]:
import scanpy as sc
import numpy as np
import functools
import jax
from cfp.metrics import compute_metrics, compute_mean_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp
import scanpy as sc
import gseapy as gp
import matplotlib.pyplot as plt
import gseapy as gp
import anndata as ad
import decoupler
from pathlib import Path

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/optuna/study/_optimize.py:29: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [2]:
split = 4

In [3]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [4]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)


In [5]:
adata_ood.obs.head()

,sample,Size_Factor,n.umi,RT_well,Drug1,Drug2,Well,n_genes,n_genes_by_counts,total_counts,...,split,control,cell_type,cell_line,smiles_drug_1,smiles_drug_2,ood_1,ood_2,ood_3,ood_4
Cell,,,,,,,,,,,,,,,,,,,,,
A01_A02_RT_BC_13_Lig_BC_1,sciPlex_theis,1.083277,2908,RT_13,Cediranib,PCI-34051,B1,1896,1895,2907.0,...,ood,0,A549,A549,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,COC1=CC=C(C=C1)CN2C=CC3=C2C=C(C=C3)C(=O)NO,not ood,not ood,not ood,Cediranib+PCI-34051
A01_A02_RT_BC_13_Lig_BC_15,sciPlex_theis,0.946936,2542,RT_13,Cediranib,PCI-34051,B1,1655,1652,2539.0,...,ood,0,A549,A549,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,COC1=CC=C(C=C1)CN2C=CC3=C2C=C(C=C3)C(=O)NO,not ood,not ood,not ood,Cediranib+PCI-34051
A01_A02_RT_BC_13_Lig_BC_20,sciPlex_theis,1.765727,4740,RT_13,Cediranib,PCI-34051,B1,2823,2821,4738.0,...,ood,0,A549,A549,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,COC1=CC=C(C=C1)CN2C=CC3=C2C=C(C=C3)C(=O)NO,not ood,not ood,not ood,Cediranib+PCI-34051
A01_A02_RT_BC_13_Lig_BC_36,sciPlex_theis,1.394328,3743,RT_13,Cediranib,PCI-34051,B1,2298,2296,3741.0,...,ood,0,A549,A549,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,COC1=CC=C(C=C1)CN2C=CC3=C2C=C(C=C3)C(=O)NO,not ood,not ood,not ood,Cediranib+PCI-34051
A01_A02_RT_BC_13_Lig_BC_39,sciPlex_theis,0.885844,2378,RT_13,Cediranib,PCI-34051,B1,1638,1637,2377.0,...,ood,0,A549,A549,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,COC1=CC=C(C=C1)CN2C=CC3=C2C=C(C=C3)C(=O)NO,not ood,not ood,not ood,Cediranib+PCI-34051


In [6]:
adata_ood.obs["condition"].cat.categories

Index(['Cediranib+PCI-34051', 'Givinostat+Crizotinib', 'Givinostat+SRT2104',
       'Givinostat+Sorafenib', 'control', 'control+Givinostat',
       'control+SRT2104'],
      dtype='object')

In [7]:
adata_train.obs["condition"].cat.categories

Index(['Alvespimycin+Pirarubicin', 'Dacinostat+Danusertib',
       'Dacinostat+Dasatinib', 'Dacinostat+PCI-34051', 'Givinostat+Carmofur',
       'Givinostat+Cediranib', 'Givinostat+Curcumin', 'Givinostat+Dasatinib',
       'Givinostat+SRT1720', 'Givinostat+Tanespimycin',
       'Panobinostat+Alvespimycin', 'Panobinostat+Crizotinib',
       'Panobinostat+Curcumin', 'Panobinostat+Dasatinib',
       'Panobinostat+PCI-34051', 'Panobinostat+SRT1720',
       'Panobinostat+SRT2104', 'Panobinostat+SRT3025',
       'Panobinostat+Sorafenib', 'SRT2104+Alvespimycin', 'SRT3025+Cediranib',
       'control', 'control+Alvespimycin', 'control+Dacinostat',
       'control+Dasatinib', 'control+Panobinostat'],
      dtype='object')

In [8]:
control_mean = adata_ood[adata_ood.obs["condition"]=="control"].X.mean(axis=0)
rng = np.random.default_rng(0)
drug_effect = {}
for cond in adata_train.obs["condition"].cat.categories:
    if cond=="control":
        continue
    drug_effect[cond] = adata_train[adata_train.obs["condition"]==cond].X.mean(axis=0) - control_mean

drug_effect["control"] = adata_ood[adata_ood.obs["condition"]=="control"].X.toarray()
    

In [9]:
drug_effect_ood = {}

drug_effect_ood['Cediranib+PCI-34051'] = (
    drug_effect['control'] 
    + drug_effect['Dacinostat+PCI-34051'] 
    - drug_effect['control+Dacinostat']
    + drug_effect['Givinostat+Cediranib']
    - drug_effect['Givinostat+Dasatinib']
    + drug_effect['control+Dasatinib']
)


drug_effect_ood['Givinostat+Crizotinib'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Dasatinib'] 
    - drug_effect['control+Dasatinib']
    + drug_effect['Panobinostat+Crizotinib']
    - drug_effect['control+Panobinostat']
)

drug_effect_ood['Givinostat+SRT2104'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Dasatinib'] 
    - drug_effect['control+Dasatinib']
    + drug_effect['Panobinostat+SRT2104']
    - drug_effect['control+Panobinostat']
)

drug_effect_ood['Givinostat+Sorafenib'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Dasatinib'] 
    - drug_effect['control+Dasatinib']
    + drug_effect['Panobinostat+Sorafenib']
    - drug_effect['control+Panobinostat']
)

drug_effect_ood['control+Givinostat'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Dasatinib'] 
    - drug_effect['control+Dasatinib']
)


drug_effect_ood['control+SRT2104'] = (
    drug_effect['control'] 
    + drug_effect['Panobinostat+SRT2104'] 
    - drug_effect['control+Panobinostat']
)

In [10]:
import anndata as ad
import pandas as pd
all_data = []
conditions = []

for condition, array in drug_effect_ood.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_pred_ood = ad.AnnData(X=all_data_array, obs=obs_data)

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [11]:
adata_pred_ood.X = np.array(adata_pred_ood.X)

In [12]:
adata_pred_ood.write(f"/lustre/groups/ml01/workspace/ot_perturbation/models/additive/combosciplex/adata_ood_with_predictions_{split}.h5ad")

In [13]:
drug_effect_ood['control+SRT2104']

matrix([[ 9.5985532e-03,  5.6056678e-03,  9.3820125e-02, ...,
          1.1694458e-03, -3.0335565e-03, -6.0997233e-03],
        [ 9.5985532e-03,  5.6056678e-03,  9.3820125e-02, ...,
          1.1694458e-03, -3.0335565e-03, -6.0997233e-03],
        [ 9.5985532e-03,  5.6056678e-03,  9.3820125e-02, ...,
          1.1694458e-03, -3.0335565e-03, -6.0997233e-03],
        ...,
        [ 3.6023989e+00,  5.6056678e-03,  9.3820125e-02, ...,
          1.1694458e-03, -3.0335565e-03, -6.0997233e-03],
        [ 9.5985532e-03,  5.6056678e-03,  9.3820125e-02, ...,
          1.1694458e-03, -3.0335565e-03, -6.0997233e-03],
        [ 9.5985532e-03,  5.6056678e-03,  9.3820125e-02, ...,
          1.1694458e-03, -3.0335565e-03, -6.0997233e-03]], dtype=float32)